## Sparse Grid Regression with the Combination Technique: 


### Utility functions

In [ ]:
from sparseSpACE.Function import *
from sparseSpACE.Utils import *
import numpy as np
import math

def construct_dataset(dim, function, num_points):
    #one_d_grid = np.linspace(0, 1, points_per_dim)
    #grid_arr = [one_d_grid]*dim
    grid_points = np.random.rand(num_points,2)
    #grid_points = get_cross_product_numpy_array(grid_arr)
    y_vals = np.array([function(x) for x in grid_points])
    return grid_points, y_vals.flatten()

    
#def split_dataset(data, targets, ratio):
#    training_size = 0
#    test_size = 0
#    for i in range(len(data)):
#        if i%ratio != 0:
#            training_size += 1
#        else:
#            test_size += 1
#            
#    training_data = np.zeros((training_size, len(data[0]))) 
#    training_targets = np.zeros(training_size) 
#    
#    test_data = np.zeros((test_size, len(data[0]))) 
#    test_targets = np.zeros(test_size) 
#    
#    training_index = 0
#    test_index = 0
#    
#    for i in range(len(data)):
#        if i%ratio != 0:
#            training_data[training_index] = data[i]
#            training_targets[training_index] = targets[i]
#            training_index += 1
#        else:
#            test_data[test_index] = data[i]
#            test_targets[test_index] = targets[i]
#            test_index += 1
#    
#    return training_data, training_targets, test_data, test_targets


#def train_regression(training_data, training_targets, regularization, matrix, minimum_level, maximum_level):
#    dim = len(training_data[0])
#    
#    a = np.zeros(dim)
#    b = np.ones(dim)
#    
#    operation = Regression(training_data, training_targets, regularization, dim, regularization_matrix=matrix)
#    
#    combiObject = StandardCombi(a, b, operation=operation)
#    
#    combiObject.perform_operation(minimum_level, maximum_level)
#    
#    return operation, combiObject


#def test_regression(test_data, test_targets, combiObject):
#    learned_targets = combiObject(test_data)
#    
#    difference = 0
#    for i in range(len(learned_targets)):
#        difference += (test_targets[i] - learned_targets[i]) ** 2
#        
#    return math.sqrt(difference/len(test_targets))

#def scale_data(data, target, rangee=[0.05,0.95]):
#    dataSet = DataSetRegression((data, target))
#    dataSet.scale_range(rangee)
#    data, target = dataSet.get_data()[0], dataSet.get_data()[1]
#    return data, target

### Version without spatial adaptivity:

In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.DEMachineLearning import *
from sklearn import datasets

######################################## Choose data set####################################
# -------------------------------------- Dataset 0 (Gaussian) --------------------------------------------
func = GenzGaussian((0.5,0.5), (10,10))
data, target = construct_dataset(2, func, 200)
# -------------------------------------- Dataset 0 (Gaussian) --------------------------------------------
    
    
# -------------------------------------- Dataset 0 (Stern) --------------------------------------------
# func = GenzC0((10,10), (0.5,0.5))
# data, target = construct_dataset(2, func, 500)
# -------------------------------------- Dataset 0 (Stern) --------------------------------------------
    
# -------------------------------------- Dataset 1 (Boston) --------------------------------------------
# data, target = datasets.load_boston(return_X_y=True)
# -------------------------------------- Dataset 1 (Boston) -------------------------------------------

# -------------------------------------- Dataset 2 (Diabetes) --------------------------------------------
# data, target = datasets.load_diabetes(return_X_y=True)
# -------------------------------------- Dataset 2 (Diabetes) --------------------------------------------
#############################################################################################

row = "["
for i in range(len(data)):
    row += "["
    for j in range(len(data[i])):
        row += str(data[i][j])
        if j != len(data[i])-1:
            row += ", "
    row += "]"
    if i != len(data)-1:
        row += ", "
    row += "\n"
row += "]"
print(row)

print(", ")

row = "["
for j in range(len(target)):
    row += str(target[j])
    if j != len(target)-1:
        row += ", "
row += "]"
print(row)

operation = Regression(data, target, 0, 'C')


levelvec = [(1, 4), (2, 3), (3, 2), (4, 1), (1, 3), (2, 2), (3, 1)]

for i in range(len(levelvec)):
    operation.grid.setCurrentArea(np.zeros(len(levelvec[i])), np.ones(len(levelvec[i])), levelvec[i])

    alpha = operation.solve_regression(levelvec[i])
    print(levelvec[i], ":")
    line = "["
    for j in range(len(alpha)):
        line += str(alpha[j])
        if j != len(alpha)-1:
            line += ", "
    line += "]"
    if i != len(levelvec)-1:
        line += ", "
    
    print(line)


operation.plot_dataset()

## Version with the spatial adaptivity:

In [ ]:
def test_regression_spatially_adaptive(test_data, test_targets, adaptiveCombiInstanceSingleDim, grid):
    learned_targets = adaptiveCombiInstanceSingleDim(test_data)
    
    difference = 0
    for i in range(len(learned_targets)):
        difference += (test_targets[i] - learned_targets[i]) ** 2
        
    return math.sqrt(difference/len(test_targets))

In [ ]:
%matplotlib inline
import sparseSpACE
import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *

# dimension of the problem
dim = 2

# define integration domain boundaries
a = np.zeros(dim)
b = np.ones(dim)

#################################### Choose Data set ########################
# ---------------------------------- Dataset 0 (Gaussian) -------------------
#func = GenzGaussian((0.5,0.5), (10,10))
#data, target = construct_dataset(dim, func, 500)
# ---------------------------------- Dataset 0 (Gaussian) -------------------

# ---------------------------------- Dataset 1 (Linear) ---------------------
#func = FunctionLinear((1,1))
#data, target = construct_dataset(dim, func, 500)
# ---------------------------------- Dataset 1 (Linear) ---------------------

# ---------------------------------- Dataset 2 (Boston) dimension = 13 ------
data, target = datasets.load_boston(return_X_y=True)
# -------------------------------------- Dataset 2 (Boston) -----------------

# ---------------------------------- Dataset 3 (Diabetes) dimension = 10 ----
# data, target = datasets.load_diabetes(return_X_y=True)
# ---------------------------------- Dataset 3 (Diabetes) -------------------
############################################################################


# define error estimator for refinement
#errorOperator = ErrorCalculatorSingleDimVolumeGuided()
# define equidistant grid
#grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=False)
# define operation which shall be performed in the combination technique
#operation = Regression(training_data, training_targets, 0.00000001, dim, regularization_matrix=1, grid=grid)
# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
#adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, margin=0.5, operation=operation, rebalancing=False)

operation = Regression(data, target, 10**-6, 'I')
adaptiveCombiInstanceSingleDim = operation.train_spatially_adaptive(percentage_of_testdata=0.2, margin=0.3, tolerance=10**-3, max_evaluations=500)
difference = operation.test_spatially_adaptive(adaptiveCombiInstanceSingleDim)
print("Testfehler: ")
print(difference)

# performing the spatially adaptive refinement with the SingleDim method
#adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1, 2, errorOperator, 10**-3, do_plot=False, max_evaluations=500)
#difference = test_regression_spatially_adaptive(test_data, test_targets, adaptiveCombiInstanceSingleDim, operation.grid)
#print("Testfehler:")
#print(difference)